In [1]:
#include <iostream>
#include <vector>
#include <string>
#include <sstream>
#include <algorithm>    // std::reverse
#include <iterator>
#include <bitset>

using namespace std;

In [2]:
auto sparse_hash(vector<int> lengths, int list_size, int nrounds) {
    int pos = 0;
    int skip = 0;
    vector <int> l;
    l.reserve(list_size);
    for (int i = 0; i < list_size; i++) {
        l.push_back(i);
    }

    for (int i = 0; i < nrounds; i++) {
        for (auto it=lengths.begin(); it!=lengths.end(); ++it) {

            if (*it > list_size) {
                continue;
            }
            // rotate so pos is at start
            // this helps deal with list circularity
            rotate(l.begin(), l.begin()+pos, l.end());
            reverse(l.begin(), l.begin()+*it);
            // rotate back
            rotate(l.begin(), l.begin()+(list_size-pos), l.end());
            pos = (pos + *it + skip) % list_size;
            skip++;
        }
    }
    return l;
}

In [3]:
auto dense_hash(vector <int> l) {
    if (l.size() != 256)
        cerr << "List size is not 256!!\n";

    vector <int> dense_hash;
    dense_hash.reserve(16);
    // do the bitwise xor of segments of length 16
    for (int j = 0; j < 16; j++) {
        int bitxor = l[j*16];
        for (int i = 1; i < 16; i++) {
            bitxor ^= l[j*16 + i];
        }
        dense_hash.push_back(bitxor);
    }

    string hexstring = "";
    for (auto it = dense_hash.begin(); it!=dense_hash.end(); ++it) {
        stringstream sstream;
        sstream << hex << *it;
        if (sstream.str().size() < 2) {
            hexstring += '0' + sstream.str(); // pad with leading 0
        } else {
            hexstring += sstream.str();
        }
    }
    return hexstring;
}

In [4]:
auto ascii(string s) {
    vector <int> codes;
    codes.reserve(s.size()+5);
    for (auto it = s.begin(); it!=s.end(); ++it) {
        codes.push_back( (int) *it);
    }

    vector <int> extras = {17, 31, 73, 47, 23};
    copy(extras.begin(), extras.end(), back_inserter(codes));
    return codes;
}

In [5]:
auto knot_hash(string s) {
    vector <int> codes = ascii(s);
    return dense_hash(sparse_hash(codes, 256, 64));
}

In [6]:
string k = knot_hash("flqrgnkx-0")

(std::string &) "d4f76bdcbf838f8416ccfa8bc6d1f9e6"


In [7]:
auto hex_to_bin(char s) {
    stringstream ss;
    ss << hex << s;
    unsigned n;
    ss >> n;
    bitset<4> b(n);
    return b.to_string();
}

In [8]:
auto squares(string key) {
    unsigned squares = 0;
    for (auto i = 0; i < 128; i++) {
        string bin = "";
        string k = knot_hash(key + "-" + to_string(i));
        for (auto it = k.begin(); it!=k.end(); ++it) {
            string s = hex_to_bin(*it);
            bin += hex_to_bin(*it);
        }
        // add up squares 
        for (auto it = bin.begin(); it!=bin.end(); ++it) {
            squares += int(*it) - int('0');
        }
    }
    return squares;
}

In [9]:
squares("flqrgnkx")

(unsigned int) 8108


In [10]:
squares("oundnydw")

(unsigned int) 8106
